In [6]:
from solve import search

In [7]:
with open('beispieldaten/kaese8.txt', 'r') as f:
    stack = [tuple(map(int, f.readline().split())) for _ in range(int(f.readline()))]

In [8]:
for path, size, n_virtual in search(stack):
    print(f'{path=}, {size=}, {n_virtual=}')
# a combination of multiple of these partial solutions should (only nodes in the end can be removed) use up all slices
# todo maybe disallow permutations of the same slice
# 'virtual' slices are included in the path, but not counted towards the number of elements and always added/removed in conjunction with a real next slice
# for a given size choose the one with the least virtual slices (maybe bfs)

path=(0,), size=(1, 2, 1), n_virtual=0
path=(1, 0, (1, 2), 2), size=(2, 2, 5), n_virtual=1
path=(2, 0), size=(1, 2, 6), n_virtual=0


In [9]:
# initialpath + rest(stack - initialpath) -> unpath (gets removed from the end of initial path)
# nodes: set of nodes available, stock: list of previous solution in order of possible removal -> list of solutions accepted
# len of stock is len(stack) - len(nodes)
def make_path(nodes: set, stock: list or None) -> list(path):
    pass